In [1]:
import glob
from PIL import Image,ImageOps
import numpy as np
from scipy.misc import imresize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [2]:
!dir
print("--------------------------------------------------------------")
!dir 'D:\MyPython\SeedlingClassification\'

 Volume in drive D is DATA
 Volume Serial Number is 2E14-EA95

 Directory of D:\MyPython\SeedlingClassification

02/04/2018  12:16 AM    <DIR>          .
02/04/2018  12:16 AM    <DIR>          ..
02/03/2018  06:14 PM    <DIR>          .ipynb_checkpoints
02/03/2018  07:12 PM           348,552 model.h5
02/04/2018  12:10 AM            24,456 results.csv
11/21/2017  02:37 PM            19,863 sample_submission.csv
02/04/2018  12:16 AM           882,928 Seedling Classification.ipynb
02/03/2018  06:11 PM    <DIR>          test
02/03/2018  06:14 PM    <DIR>          train
               4 File(s)      1,275,799 bytes
               5 Dir(s)  678,856,744,960 bytes free
--------------------------------------------------------------


The filename, directory name, or volume label syntax is incorrect.


In [6]:
fns = glob.glob(r'train/*/*.*')
fns[0:5]

['train\\Black-grass\\0050f38b3.png',
 'train\\Black-grass\\0183fdf68.png',
 'train\\Black-grass\\0260cffa8.png',
 'train\\Black-grass\\05eedce4d.png',
 'train\\Black-grass\\075d004bc.png']

In [4]:
labels = []
fitted_imgs=[]
for fn in fns:
    if fn[-3:]!='png':
        continue
    labels.append(fn.split('\\')[-2])
    fitted_imgs.append(
        ImageOps.fit(Image.open(fn),(48,48),Image.ANTIALIAS).convert('RGB'))

In [5]:
imgs_array = np.array([np.array(fitted_img) 
                       for fitted_img in fitted_imgs])/255

In [6]:
lb = LabelBinarizer().fit(labels)
lb_array = lb.transform(labels)

In [7]:
trainX, validX, trainY, validY = train_test_split(imgs_array,lb_array
                                                 ,test_size = 0.05
                                                  ,random_state=42)

In [8]:
from keras.layers import Dropout, Input, Dense,Activation,GlobalMaxPooling2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model, load_model
from keras.optimizers import Adam

Using TensorFlow backend.


In [9]:
IM_input = Input((48, 48, 3))
IM = Conv2D(16, (3, 3))(IM_input)
IM = BatchNormalization(axis = 3)(IM)
IM = Activation('relu')(IM)
IM = Conv2D(16, (3, 3))(IM)
IM = BatchNormalization(axis = 3)(IM)
IM = Activation('relu')(IM)
IM = MaxPooling2D((2, 2), strides=(2, 2))(IM)
IM = Conv2D(32, (3, 3))(IM)
IM = BatchNormalization(axis = 3)(IM)
IM = Activation('relu')(IM)
IM = Conv2D(32, (3, 3))(IM)
IM = BatchNormalization(axis = 3)(IM)
IM = Activation('relu')(IM)
IM = GlobalMaxPooling2D()(IM)

IM = Dense(64, activation='relu')(IM)
IM = Dropout(0.5)(IM)
IM = Dense(32, activation='relu')(IM)
IM = Dropout(0.5)(IM)
IM = Dense(12, activation='softmax')(IM)
model = Model(inputs=IM_input, outputs=IM)
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4), metrics=['acc'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 46, 16)        448       
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 46, 46, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 16)        2320      
_________________________________________________________________
batch_normalization_2 (Batch (None, 44, 44, 16)        64        
_________________________________________________________________
activation_2 (Activation)    (None, 44, 44, 16)        0         
__________

In [10]:
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.callbacks import ModelCheckpoint

In [11]:
batch_size = 64
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
earlystop = EarlyStopping(patience=10)
modelsave = ModelCheckpoint(
    filepath='model.h5', save_best_only=True, verbose=1)
model.fit(
    trainX, trainY, batch_size=batch_size,
    epochs=200, 
    validation_data=(validX, validY),
    callbacks=[annealer, earlystop, modelsave]
)

Train on 4512 samples, validate on 238 samples
Epoch 1/200
4480/4512 [============================>.] - ETA: 0s - loss: 2.6927 - acc: 0.0991 

In [25]:
z = glob.glob('test//*.png')

test_imgs = []
names = []
for fn in z:
    if fn[-3:] != 'png':
        continue
    names.append(fn.split('\\')[-1])
    new_img = Image.open(fn)
    test_img = ImageOps.fit(new_img, (48, 48), Image.ANTIALIAS).convert('RGB')
    test_imgs.append(test_img)
model = load_model('model.h5')

In [26]:
timgs = np.array([np.array(im) for im in test_imgs])
testX = timgs.reshape(timgs.shape[0], 48, 48, 3) / 255

In [27]:
yhat = model.predict(testX)
test_y = lb.inverse_transform(yhat)

In [28]:
import pandas as pd
df = pd.DataFrame(data={'file': names, 'species': test_y})
df_sort = df.sort_values(by=['file'])
df_sort.to_csv('results.csv', index=False)

# ------------------------------------

In [15]:
fns = glob.glob(r'train/*/*.*')
fns[0:5]

['train\\Black-grass\\0050f38b3.png',
 'train\\Black-grass\\0183fdf68.png',
 'train\\Black-grass\\0260cffa8.png',
 'train\\Black-grass\\05eedce4d.png',
 'train\\Black-grass\\075d004bc.png']

In [21]:
for fn in fns:
    print(fn.split('\\')[-2])

Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Black-grass
Blac

In [25]:
labels = []
labels = [fn.split('\\')[-2] for fn in fns]

In [18]:
list(labels)

[<generator object <genexpr> at 0x000001F3BB2D0DB0>]

In [35]:
lb = LabelBinarizer()

In [36]:
lb.fit_transform(labels)

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])